# 1. Скачайте тренировочный датасет. Распакуйте один файл и посмотрите на его структуру.

In [1]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install catboost

   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/101.1 MB 787.7 kB/s eta 0:02:09
   ---------------------------------------- 0.2/101.1 MB 1.6 MB/s eta 0:01:04
   ---------------------------------------- 0.5/101.1 MB 2.9 MB/s eta 0:00:36
   ---------------------------------------- 0.7/101.1 MB 3.6 MB/s eta 0:00:28
    --------------------------------------- 1.5/101.1 MB 5.8 MB/s eta 0:00:18
   - -------------------------------------- 3.3/101.1 MB 11.0 MB/s eta 0:00:09
   -- ------------------------------------- 6.6/101.1 MB 19.3 MB/s eta 0:00:05
   --- ------------------------------------ 8.2/101.1 MB 20.8 MB/s eta 0:00:05
   --- ------------------------------------ 8.8/101.1 MB 21.7 MB/s eta 0:00:05
   ---- ----------------------------------- 10.5/101.1 MB 29.8 MB/s eta 0:00:04
   ----- ---------------------------------- 13.3/101.1 MB 46.7 MB/s et

In [4]:
pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 320.0 kB/s eta 0:06:31
   ---------------------------------------- 0.0/124.9 MB 393.8 kB/s eta 0:05:18
   ---------------------------------------- 0.1/124.9 MB 901.1 kB/s eta 0:02:19
   ---------------------------------------- 0.4/124.9 MB 2.1 MB/s eta 0:01:00
   ---------------------------------------- 0.7/124.9 MB 3.1 MB/s eta 0:00:40
   ---------------------------------------- 1.3/124.9 MB 4.7 MB/s eta 0:00:27
    --------------------------------------- 2.8/124.9 MB 8.4 MB/s eta 0:00:15
   - -------------------------------------- 5.8/124.9 MB 15.5 MB/s eta 0:00:08
   -- ------------------------------------- 8.7/124.9 MB 21.3 MB/s eta 0:00:06
   -- ------------------------------------- 8.7/124.9 MB 21.3 MB/s eta 0:00:06
   -- ------------------------------------- 8.7/124.9 MB 21.3 MB/s eta 0:00:06
   ---- ----------------------------------- 13.6/124.9 MB 46.

In [5]:
pip install lightgbm 

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.4 MB 825.8 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.4 MB 1.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.4/1.4 MB 2.8 MB/s eta 0:00:01
   -------------------- ------------------- 0.7/1.4 MB 3.6 MB/s eta 0:00:01
   ----------------------------------- ---- 1.3/1.4 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 5.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import dill as dill
import numpy as np
import warnings

from os import listdir
from os.path import isfile, join
from collections import Counter

from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_auc_score

warnings.filterwarnings("ignore")

In [7]:
# Откроем первый паркет для ознакомления с данными, присвом переменную df0
df0 = pd.read_parquet('train_data/train_data_0.pq')
# Выведем 5 строк датасета паркета
df0.head()

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,...,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,...,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,...,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,...,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,...,3,3,3,4,1,3,4,1,0,0


In [8]:
# Выведем информацию о датасете
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1974724 entries, 0 to 1974723
Data columns (total 61 columns):
 #   Column                         Dtype
---  ------                         -----
 0   id                             int64
 1   rn                             int64
 2   pre_since_opened               int64
 3   pre_since_confirmed            int64
 4   pre_pterm                      int64
 5   pre_fterm                      int64
 6   pre_till_pclose                int64
 7   pre_till_fclose                int64
 8   pre_loans_credit_limit         int64
 9   pre_loans_next_pay_summ        int64
 10  pre_loans_outstanding          int64
 11  pre_loans_total_overdue        int64
 12  pre_loans_max_overdue_sum      int64
 13  pre_loans_credit_cost_rate     int64
 14  pre_loans5                     int64
 15  pre_loans530                   int64
 16  pre_loans3060                  int64
 17  pre_loans6090                  int64
 18  pre_loans90                    int64
 19  

# Датасет загружен, его структура изучена.

# 2. Подумайте, какие признаки могут быть полезны, и смоделируйте их. Дайте комментарии, почему вы сгенерировали тот или иной признак.

# Список признаков:

# Рассмотрим каждый признак на распределение, методом value_counts()

In [9]:
pre_since_opened = "Количество дней с даты открытия до даты сбора данных"
pre_since_confirmed = "Количество дней с даты подтверждения информации по кредиту до даты сбора данных"
pre_pterm = "Плановое количество дней с даты открытия кредита до даты закрытия"
pre_fterm = "Фактическое количество дней с даты открытия кредита до даты закрытия"
pre_till_pclose = "Плановое количество дней с даты сбора данных до даты закрытия кредита"
pre_till_fclose = "Фактическое количество дней с даты сбора данных до даты закрытия кредита"
pre_loans_credit_limit = "Кредитный лимит"
pre_loans_next_pay_summ = "Сумма следующего платежа по кредиту"
pre_loans_outstanding = "Оставшаяся невыплаченная сумма кредита"
pre_loans_total_overdue = "Текущая просроченная задолженность"
pre_loans_max_overdue_sum = "Максимальная просроченная задолженность"
pre_loans_credit_cost_rate = "Полная стоимость кредита"
pre_loans5 = "Число просрочек до 5 дней"
pre_loans530 = "Число просрочек от 5 до 30 дней"
pre_loans3060 = "Число просрочек от 30 до 60 дней"
pre_loans6090 = "Число просрочек от 60 до 90 дней"
pre_loans90 = "Число просрочек более чем на 90 дней"
is_zero_loans5 = "Флаг: нет просрочек до 5 дней"
is_zero_loans530 = "Флаг: нет просрочек от 5 до 30 дней"
is_zero_loans3060 = "Флаг: нет просрочек от 30 до 60 дней"
is_zero_loans6090 = "Флаг: нет просрочек от 60 до 90 дней"
is_zero_loans90 = "Флаг: нет просрочек более чем на 90 дней"
pre_util = "Отношение оставшейся невыплаченной суммы кредита к кредитному лимиту"
pre_over2limit = "Отношение текущей просроченной задолженности к кредитному лимиту"
pre_maxover2limit = "Отношение максимальной просроченной задолженности к кредитному лимиту"
is_zero_util = "Флаг: отношение оставшейся невыплаченной суммы кредита к кредитному лимиту равно 0"
is_zero_over2limit = "Флаг: отношение текущей просроченной задолженности к кредитному лимиту равно 0"
is_zero_maxover2limit = "Флаг: отношение максимальной просроченной задолженности к кредитному лимиту равно 0"
enc_paym_1= "Статусы ежемесячных платежей за последние N месяцев"
enc_loans_account_holder_type = "Тип отношения к кредиту"
enc_loans_credit_status = "Статус кредита"
enc_loans_account_cur = "Валюта кредита"
enc_loans_credit_type = "Тип кредита"
pclose_flag = "Плановое количество дней с даты открытия кредита до даты закрытия не определено"
fclose_flag = "Флаг: фактическое количество дней с даты открытия кредита до даты закрытия не определено"
print(pre_since_opened, df0['pre_since_opened'].value_counts())
print(pre_since_confirmed, df0['pre_since_confirmed'].value_counts())
print(pre_pterm, df0['pre_pterm'].value_counts())
print(pre_fterm, df0['pre_fterm'].value_counts())
print(pre_till_fclose, df0['pre_till_fclose'].value_counts())
print(pre_loans_next_pay_summ, df0['pre_loans_next_pay_summ'].value_counts())
print(pre_loans_outstanding, df0['pre_loans_outstanding'].value_counts())
print(pre_loans_credit_limit, df0['pre_loans_credit_limit'].value_counts())
print(pre_till_pclose, df0['pre_till_pclose'].value_counts())
print(pre_loans_total_overdue, df0['pre_loans_total_overdue'].value_counts())
print(pre_loans_max_overdue_sum, df0['pre_loans_max_overdue_sum'].value_counts())
print(pre_loans_credit_cost_rate, df0['pre_loans_credit_cost_rate'].value_counts())
print(pre_loans5, df0['pre_loans5'].value_counts()) # признаки pre_loans похожи друг на друга. рассмотрим только один из этой группы
print(is_zero_loans5, df0['is_zero_loans5'].value_counts()) # признаки is_zero_loans похожи друг на друга. рассмотрим только один из этой группы
print(pre_util, df0['pre_util'].value_counts())
print(pre_over2limit, df0['pre_over2limit'].value_counts())
print(pre_maxover2limit, df0['pre_maxover2limit'].value_counts())
print(is_zero_util, df0['is_zero_util'].value_counts())
print(is_zero_over2limit, df0['is_zero_over2limit'].value_counts())
print(is_zero_maxover2limit, df0['is_zero_maxover2limit'].value_counts())
print(enc_paym_1, df0['enc_paym_1'].value_counts()) # признаки is_zero_loans похожи друг на друга. рассмотрим только один из этой группы
print(enc_loans_account_holder_type, df0['enc_loans_account_holder_type'].value_counts())
print(enc_loans_credit_status, df0['enc_loans_credit_status'].value_counts())
print(enc_loans_account_cur, df0['enc_loans_account_cur'].value_counts())
print(enc_loans_credit_type, df0['enc_loans_credit_type'].value_counts())
print(pclose_flag, df0['pclose_flag'].value_counts())
print(fclose_flag, df0['fclose_flag'].value_counts())

Количество дней с даты открытия до даты сбора данных pre_since_opened
18    159072
6     154230
13    141904
4     138266
8     133032
0     111331
7      96090
19     94237
2      91807
10     91017
9      90118
17     86449
1      85775
11     82871
3      82231
5      80730
14     78560
15     73219
12     71609
16     32176
Name: count, dtype: int64
Количество дней с даты подтверждения информации по кредиту до даты сбора данных pre_since_confirmed
9     418663
6     169181
14    134266
3     123579
17    116876
4     106465
8     101021
7      99650
13     84393
10     83769
0      80598
12     79820
2      77607
1      76836
16     75548
5      74440
11     71962
15        50
Name: count, dtype: int64
Плановое количество дней с даты открытия кредита до даты закрытия pre_pterm
4     374796
14    152466
7     126906
2     121765
16    110465
17    109905
8     109871
9     109775
1     108185
13    105364
15    102845
11     94518
12     90278
6      83052
0      77813
3      69856


# Сделаем выводы
1. Признак текущая просроченная задолженность pre_loans_total_overdue, имеет значение 0, соответсвенно является бесполезным при обучении модели, его можно удалить;
2. Все признаки являются категориальными и мы смело можем их кодировать в ohe;
3. Из признака pre_loans_credit_limit можно создать фичу growth_limit - увеличился ли кредитный лимит по сравнению с первым кредитом?
4. Учитывая что у одного id клиента может множество кредитов, обучить модель не получиться, необходимо признак rn  агрегировать по count, таким образом получим количество кредитов у одного id;
5. С помощью is_zero_util можно определить количество закрытых кредитов. Создадим фичу количество открытых кредитов open_loans, если is_zero_util = 0, значит кредит не закрыт;
6. Инвертируем значения признаков is_zero_loans - так как нам важно наличие именно просрочек, а не их отсутсвие;

# Создадим ряд вспомогательных функций

In [10]:
# Функции сохранения - загрузки моделей
def dill_dump(path, model):
    with open(path, 'wb') as file:
        dill.dump(model, file)

def dill_load(path):
    with open(path, 'rb') as file:
        model = dill.load(file)
    return model

# Функция кодирования ohe и агрегирования по 1 признаку
def convert_one_features(df0=pd.DataFrame, column=str, col_id=False) -> pd.DataFrame:
    df_col = df0[['id', column]]
    ohe = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

    df_col[ohe.get_feature_names_out()] = ohe.fit_transform(df_col[[column]])
    df_col = df_col.drop(columns=column)
    df_col = df_col.groupby(['id']).agg('sum').reset_index()
    if col_id == False:
        df_col = df_col.drop(columns='id')
    return df_col

# Функция агрегирования rn по count (получим в итоговом датасете количество кредитов на один id)
def agg_rn(df=pd.DataFrame) -> pd.DataFrame:
    df_agg_rn = df[['id', 'rn']].copy()
    df_agg_rn = df_agg_rn.groupby(['id']).agg('count').reset_index()
    return df_agg_rn

# Функция для удаления признака
def del_feature(df=pd.DataFrame, col=str) -> pd.DataFrame:
    df_new = df.copy()
    return df_new.drop(columns=col)

# Функция для инвертирования признаков
def invert(df=pd.DataFrame, columns=list, col_id=False) -> pd.DataFrame: 
    df_invert = df[['id'] + columns].copy()
    for col in columns:
        df_invert[col] = df_invert[col].apply(lambda x: 1 if x==0 else 0) #делаем реверс значений по каждой колонке
    if col_id == False:
        return df_invert.drop(columns='id')
    else:
        return df_invert
    
# Функция для получения признака - количество открытых кредитов
def open_loans_credit(df=pd.DataFrame) -> pd.Series:
    df_open = df[['id', 'is_zero_util']].copy()
    df_open = invert(df_open, ['is_zero_util'], col_id=True)
    df_open = df_open.groupby(['id']).agg('sum').reset_index()
    df_open = df_open.rename(columns={'is_zero_util':'open_loans'})
    return df_open['open_loans']

# Функция для получения growth_limit - увеличился ли кредитный лимит по сравнению с первым кредитом
def growth_limit_feature(df=pd.DataFrame) -> pd.Series:
    df_change = df[['id', 'pre_loans_credit_limit']].copy()
    df_change_first = df_change.drop_duplicates(subset=['id'], keep='first').rename(columns={
        'pre_loans_credit_limit': 'first'
    }).set_index('id')
    df_change_last = df_change.drop_duplicates(subset=['id'], keep='last').rename(columns={
        'pre_loans_credit_limit': 'last'
    }).set_index('id')
    df_change = pd.concat([df_change_first, df_change_last], axis=1).reset_index()
    df_change['growth_limit'] = df_change['last'] - df_change['first']
    return df_change['growth_limit']

# Функция для агрегирования всего датасета по 1 столбцу для экономии памяти
def convert_df(df0=pd.DataFrame) -> pd.DataFrame:
    df = pd.DataFrame()
    
    list_to_invert = [ 
        'is_zero_loans5',
        'is_zero_loans530',
        'is_zero_loans3060',
        'is_zero_loans6090',
        'is_zero_loans90'
    ]
    
    # Получаем новый агрегированный датафрейм по id с признаком rn - колич. кредитов
    df[['id', 'rn']] = agg_rn(df0).astype(np.int32)
    
    # Инвертируем необходимые признаки в переданном в функцию датасете
    df0[list_to_invert] = invert(df0, list_to_invert) 
    
    # Удаляем бесполезный признак в переданном в функцию датасете
    df0 = del_feature(df0, 'pre_loans_total_overdue') 
    
    # Добавляем к агрегированному датасету агрегированный признак - открытые кредиты
    df['open_loans'] = open_loans_credit(df0) 
    
    # Добавляем к агрегированному датасету агрегированный признак growth_limit
    df['growth_limit'] = growth_limit_feature(df0) 
    
    columns = df0.columns.to_list()[2:] # так как id и rn уже сагрегированы, то обрезаим их из списка колонок
    for column in columns:
        df_one_columns = convert_one_features(df0, column).astype(np.int8) # все фичи, кроме id и rn конвертируются в int8
        df = pd.concat([df, df_one_columns], axis=1)
    return df

# 2.1 Исследование моделей

In [11]:
# Применим созданную функцию convert_df
df = convert_df(df0)
df

,id,rn,open_loans,growth_limit,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,...,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,pclose_flag_0,pclose_flag_1,fclose_flag_0,fclose_flag_1
0,0,10,4,5,0,1,1,1,1,2,...,7,0,0,10,0,0,9,1,8,2
1,1,14,4,10,0,0,1,0,0,0,...,8,0,0,14,0,0,13,1,12,2
2,2,3,2,0,1,0,0,0,0,0,...,1,0,0,3,0,0,1,2,1,2
3,3,15,7,4,0,3,1,0,2,1,...,9,1,0,15,0,0,10,5,9,6
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,249995,13,4,14,2,1,2,0,0,0,...,7,3,0,13,0,0,12,1,10,3
249996,249996,23,5,10,0,1,2,1,2,2,...,9,3,0,23,0,0,18,5,18,5
249997,249997,7,0,5,0,2,0,0,0,0,...,2,0,0,6,1,0,6,1,5,2
249998,249998,5,4,0,0,2,0,0,0,0,...,2,0,0,5,0,0,3,2,2,3


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Columns: 398 entries, id to fclose_flag_1
dtypes: int32(2), int64(2), int8(394)
memory usage: 99.7 MB


In [15]:
target = pd.read_csv('train_target.csv')[0:250000]

X = df.drop(columns='id')
y = target['flag']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)

In [16]:
mlp = MLPClassifier(random_state=42)

mlp.fit(X_train, y_train)
predict_proba = mlp.predict_proba(X_test)[:,1]
predict = mlp.predict(X_test)
print(roc_auc_score(y_test, predict_proba))
print(precision_score(y_test, predict))
print(recall_score(y_test, predict))
print(confusion_matrix(y_test, predict))

0.5890255160469716
0.09751434034416825
0.06593406593406594
[[47509   944]
 [ 1445   102]]


# К модели встроим баланс классов

In [17]:
y_train.value_counts()

flag
0    193813
1      6187
Name: count, dtype: int64

In [18]:
os = SMOTE(random_state=42, k_neighbors=1000, sampling_strategy={0:193813, 1:193813})
X_smote, y_smote = os.fit_resample(X_train, y_train)

In [19]:
mlp = MLPClassifier(random_state=42, early_stopping=True, max_iter=1000)

mlp.fit(X_smote, y_smote)
predict_proba = mlp.predict_proba(X_test)[:,1]
predict_proba_train = mlp.predict_proba(X_train)[:,1]
predict = mlp.predict(X_test)
print(roc_auc_score(y_train, predict_proba_train))
print(roc_auc_score(y_test, predict_proba))
print(precision_score(y_test, predict))
print(recall_score(y_test, predict))
print(confusion_matrix(y_test, predict))

0.8056819728983637
0.7399921109216108
0.13793103448275862
0.002585649644473174
[[48428    25]
 [ 1543     4]]


# Выводы после обучения модели

1. Модель сильно недообучается;
2. Сделаем апсэмплинг, так как SMOTE показывает свою неэффективность на этих данных/

In [20]:
train_df = pd.concat([X_train, y_train], axis=1)
train_majority = train_df[train_df.flag==0]
train_minority = train_df[train_df.flag==1]
minority_upsampled = resample(train_minority, replace = True, n_samples = 193813, random_state = 42)
minority_upsampled["flag"].value_counts()
train_set_us = pd.concat([train_majority, minority_upsampled])
train_set_us = train_set_us.sample(frac=1)
train_set_us["flag"].value_counts()
train_up = train_set_us.drop(columns='flag')
y_up = train_set_us['flag']

In [21]:
# Создадим функцию для обучения и тестирования модели
def train_model(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    predict_proba_train = model.predict_proba(X_train)[:,1]
    predict_proba_test = model.predict_proba(X_test)[:,1]
    predict_test = model.predict(X_test)
    print(roc_auc_score(y_train, predict_proba_train))
    print(roc_auc_score(y_test, predict_proba_test))
    print(precision_score(y_test, predict_test))
    print(recall_score(y_test, predict_test))
    print(confusion_matrix(y_test, predict_test))
    return predict_proba_test, predict_test

In [22]:
mlp = MLPClassifier(random_state=42, early_stopping=True, max_iter=1000)

predict_proba_test_mlp, predict_test_mlp = train_model(train_up, X_test, y_up, y_test, mlp)

0.9966302704788217
0.636090344369198
0.08271908271908272
0.1305753070458953
[[46213  2240]
 [ 1345   202]]


In [23]:
xgb = XGBClassifier(n_jobs=-1, random_state=42)

predict_proba_test_xgb, predict_test_xgb = train_model(train_up, X_test, y_up, y_test, xgb)

0.932315885727778
0.7160021898482821
0.07569917976084593
0.4951519069166128
[[39100  9353]
 [  781   766]]


In [25]:
cat = CatBoostClassifier(iterations=10000, task_type="CPU", devices='0:1')

predict_proba_test_cat, predict_test_cat = train_model(train_up, X_test, y_up, y_test, cat)

Learning rate set to 0.015892
0:	learn: 0.6908129	total: 307ms	remaining: 51m 13s
1:	learn: 0.6886019	total: 443ms	remaining: 36m 54s
2:	learn: 0.6864199	total: 602ms	remaining: 33m 25s
3:	learn: 0.6842877	total: 747ms	remaining: 31m 7s
4:	learn: 0.6822505	total: 878ms	remaining: 29m 15s
5:	learn: 0.6802349	total: 1.03s	remaining: 28m 36s
6:	learn: 0.6783678	total: 1.18s	remaining: 28m 1s
7:	learn: 0.6765007	total: 1.3s	remaining: 27m 8s
8:	learn: 0.6746614	total: 1.44s	remaining: 26m 44s
9:	learn: 0.6728848	total: 1.58s	remaining: 26m 21s
10:	learn: 0.6711295	total: 1.72s	remaining: 26m 4s
11:	learn: 0.6694664	total: 1.87s	remaining: 25m 56s
12:	learn: 0.6678464	total: 2s	remaining: 25m 37s
13:	learn: 0.6661555	total: 2.13s	remaining: 25m 20s
14:	learn: 0.6646028	total: 2.27s	remaining: 25m 14s
15:	learn: 0.6630611	total: 2.42s	remaining: 25m 12s
16:	learn: 0.6615401	total: 2.55s	remaining: 24m 57s
17:	learn: 0.6600927	total: 2.68s	remaining: 24m 46s
18:	learn: 0.6586952	total: 2.82s	

# LGBMClassifier позволяет настраивать веса, будем использовать датасет без апсемплинга


In [26]:
lgbm = LGBMClassifier(class_weight={0:1, 1:32}, n_jobs=-1, random_state=42, reg_lambda=1000, reg_alpha=0.4)

predict_proba_test_lgbm, predict_test_lgbm = train_model(X_train, X_test, y_train, y_test, lgbm)

[LightGBM] [Info] Number of positive: 6187, number of negative: 193813
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.146077 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5702
[LightGBM] [Info] Number of data points in the train set: 200000, number of used features: 381
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.505323 -> initscore=0.021292
[LightGBM] [Info] Start training from score 0.021292
0.8299512265830655
0.7639072689224383
0.06971200309258424
0.6994182288299935
[[34014 14439]
 [  465  1082]]


# Из этих моделей лучше всех показала себя lightgbm
# Возьмем lightgbm за основную модель и обучим ее на разных датасетах с разной длинной кредитной истории (5, 8, 23)
# Для этого переделаем функцию convert_one_features и convert_df добавив в нее выбор последних n - кредитов

In [27]:
def convert_one_features(df0=pd.DataFrame, column=str, len_of_credit_history=int, col_id=False) -> pd.DataFrame:
    df_col = df0[['id', column]]
    
    ohe = OneHotEncoder(sparse=False)
    df_col = df_col.groupby(['id']).tail(len_of_credit_history).reset_index(drop=True) # выбор последних n-кредитов
    df_col[ohe.get_feature_names_out()] = ohe.fit_transform(df_col[[column]])
    df_col = df_col.drop(columns=column)
    df_col = df_col.groupby(['id']).agg('sum').reset_index()
    if col_id == False:
        df_col = df_col.drop(columns='id')
    return df_col

In [28]:
def convert_df(df0=pd.DataFrame, len_of_credit_history=int) -> pd.DataFrame:
    df = pd.DataFrame()
    
    list_to_invert = [ 
        'is_zero_loans5',
        'is_zero_loans530',
        'is_zero_loans3060',
        'is_zero_loans6090',
        'is_zero_loans90'
    ]
    
    # Получаем новый агрегированный датафрейм по id с признаком rn - колич. кредитов
    df[['id', 'rn']] = agg_rn(df0).astype(np.int32) 
    
    # Инвертируем необходимые признаки в переданном в функцию датасете
    df0[list_to_invert] = invert(df0, list_to_invert) 
    
    # Удаляем бесполезный признак в переданном в функцию датасете
    df0 = del_feature(df0, 'pre_loans_total_overdue')
    
    # Добавляем к агрегированному датасету агрегированный признак - открытые кредиты
    df['open_loans'] = open_loans_credit(df0) 
    
    # Добавляем к агрегированному датасету агрегированный признак growth_limit
    df['growth_limit'] = growth_limit_feature(df0) 
    
    # id и rn уже сагрегированы, обрежем их из списка колонок
    columns = df0.columns.to_list()[2:] 
    for column in columns:
        df_one_columns = convert_one_features(df0, column, len_of_credit_history).astype(np.int8) # все фичи, кроме id и rn конвертируются в int8
        df = pd.concat([df, df_one_columns], axis=1)
    return df

# Обучим модель в цикле на датасетах с разной кредитной историей (5,8,23)

In [29]:
dict_score = dict()

for i in [5,8,23]:
    df = convert_df(df0, i)
    target = pd.read_csv('train_target.csv')[0:250000]
    X = df.drop(columns='id')
    y = target['flag']

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)
    
    lgbm = LGBMClassifier(class_weight={0:1, 1:32}, n_jobs=-1, random_state=42, reg_lambda=1000, reg_alpha=0.4)

    predict_proba_test_lgbm, predict_test_lgbm = train_model(X_train, X_test, y_train, y_test, lgbm)
    
    dict_score[i] = predict_proba_test_lgbm
    
dict_score

[LightGBM] [Info] Number of positive: 6187, number of negative: 193813
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.095752 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2049
[LightGBM] [Info] Number of data points in the train set: 200000, number of used features: 370
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.505323 -> initscore=0.021292
[LightGBM] [Info] Start training from score 0.021292
0.822696081126443
0.7653176481367778
0.06944801026957638
0.6994182288299935
[[33955 14498]
 [  465  1082]]
[LightGBM] [Info] Number of positive: 6187, number of negative: 193813
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.117646 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2728
[Lig

{5: array([0.78524792, 0.83803598, 0.54426285, ..., 0.32998029, 0.10675994,
        0.17926516]),
 8: array([0.70844537, 0.83108123, 0.52623092, ..., 0.34755496, 0.11476635,
        0.16440979]),
 23: array([0.73395429, 0.70628072, 0.56952502, ..., 0.36672656, 0.1321142 ,
        0.13173888])}

# Выведем лучшую метрику

In [30]:
dict_roc = dict()
for k in dict_score.keys():
    dict_roc[k] = roc_auc_score(y_test, list(dict_score[k]))
max(dict_roc.values())

0.7653176481367778

# Сделаем листинг предсказаний

In [31]:
predict_proba = pd.DataFrame(dict_score).T.mean()
roc_auc_score(y_test, predict_proba.to_list())

0.7696446676859472

# Заметим что метрика значительно улучшилась
# Так как условия ТЗ по roc_auc_score выполнены и roc = 0.769644, что больше, чем 0,75 - остановимся на этом ансамбле

In [32]:
# Модуль склейки файла train из паркетов в итоговый датафрейм'''
def convert_parquet(path=str, len_of_credit_history=int) -> pd.DataFrame:
    files = [f for f in listdir(path) if isfile(join(path, f))] # создаем список файлов в директории
    
    df = pd.DataFrame()
    for file in files: # Итерируем по списку, загружаем файл из списка
        df0 = pd.read_parquet(path + file)
        df0 = convert_df(df0, len_of_credit_history)
        df = pd.concat([df, df0], axis=0)
        df[df.columns.to_list()[1:]] = df[df.columns.to_list()[1:]].fillna(0).astype(np.int8)
    return df

# Cоздадим 3 итоговых датафрейма с последними n-кредитами 5,8 и 23

In [33]:
df5 = convert_parquet('train_data/', 5)
df8 = convert_parquet('train_data/', 8)
df23 = convert_parquet('train_data/', 23)

In [34]:
df23

,id,rn,open_loans,growth_limit,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,...,enc_loans_credit_type_7,pre_loans530_8,pre_loans530_9,pre_loans_max_overdue_sum_0,pre_loans3060_0,pre_loans3060_3,pre_loans3060_4,pre_loans6090_0,pre_loans5_10,pre_loans530_17
0,0,10,4,5,0,1,1,1,1,2,...,0,0,0,0,0,0,0,0,0,0
1,1,14,4,10,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,3,2,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,15,7,4,0,3,1,0,2,1,...,0,0,0,0,0,0,0,0,0,0
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,2499995,14,2,4,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249996,2499996,5,2,-11,0,3,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
249997,2499997,14,2,-3,0,1,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
249998,2499998,7,3,-4,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
df5.isna().sum().sum()

0

In [36]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000000 entries, 0 to 249999
Columns: 417 entries, id to pre_loans530_9
dtypes: int32(1), int8(416)
memory usage: 1.2 GB


# Итоговый датафрейм создан, новые фичи сгенерированы, данные перекодированы в ohe и прошли агрегацию

# 3. После того как вы подготовили датасет с признаками, смёрджите их с целевой переменной и выведите размерность результирующего датафрейма.

In [37]:
# Сделаем сортировку по id чтобы смержить файл с target
df5 = df5.sort_values(by=['id']).reset_index()
df8 = df8.sort_values(by=['id']).reset_index()
df23 = df23.sort_values(by=['id']).reset_index()

In [38]:
df5 = df5.drop(columns='index')
df8 = df8.drop(columns='index')
df23 = df23.drop(columns='index')
df23

,id,rn,open_loans,growth_limit,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,...,enc_loans_credit_type_7,pre_loans530_8,pre_loans530_9,pre_loans_max_overdue_sum_0,pre_loans3060_0,pre_loans3060_3,pre_loans3060_4,pre_loans6090_0,pre_loans5_10,pre_loans530_17
0,0,10,4,5,0,1,1,1,1,2,...,0,0,0,0,0,0,0,0,0,0
1,1,14,4,10,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,3,2,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,15,7,4,0,3,1,0,2,1,...,0,0,0,0,0,0,0,0,0,0
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2999995,11,3,-8,1,2,0,0,2,0,...,1,0,0,0,0,0,0,0,0,0
2999996,2999996,13,4,-12,0,1,0,1,1,5,...,1,0,0,0,0,0,0,0,0,0
2999997,2999997,10,4,-14,0,0,1,1,3,0,...,0,0,0,0,0,0,0,0,0,0
2999998,2999998,5,4,9,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
target = pd.read_csv('train_target.csv').sort_values(by=['id'])
target

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2999995,2999995,0
2999996,2999996,0
2999997,2999997,0
2999998,2999998,0


In [40]:
df5['flag'] = target['flag']
df8['flag'] = target['flag']
df23['flag'] = target['flag']

In [41]:
df23

,id,rn,open_loans,growth_limit,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,...,pre_loans530_8,pre_loans530_9,pre_loans_max_overdue_sum_0,pre_loans3060_0,pre_loans3060_3,pre_loans3060_4,pre_loans6090_0,pre_loans5_10,pre_loans530_17,flag
0,0,10,4,5,0,1,1,1,1,2,...,0,0,0,0,0,0,0,0,0,0
1,1,14,4,10,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,3,2,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,15,7,4,0,3,1,0,2,1,...,0,0,0,0,0,0,0,0,0,0
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2999995,11,3,-8,1,2,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2999996,2999996,13,4,-12,0,1,0,1,1,5,...,0,0,0,0,0,0,0,0,0,0
2999997,2999997,10,4,-14,0,0,1,1,3,0,...,0,0,0,0,0,0,0,0,0,0
2999998,2999998,5,4,9,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
df23.shape

(3000000, 422)

In [43]:
df23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Columns: 422 entries, id to flag
dtypes: int32(1), int64(1), int8(420)
memory usage: 1.2 GB


# 4. С помощью лучшей модели (той, которая показала наилучшие результаты в исследованиях в предыдущем пункте) сделайте предсказания на тестовом датасете.

In [45]:
df23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Columns: 422 entries, id to flag
dtypes: int32(1), int64(1), int8(420)
memory usage: 1.2 GB


In [46]:
y = df5['flag']

In [47]:
X5 = df5.drop(columns=['id', 'flag'])

X_train5, X_test5, y_train5, y_test5 = train_test_split(X5, y, random_state=42, test_size=0.2, stratify=y)

In [48]:
X8 = df8.drop(columns=['id', 'flag'])

X_train8, X_test8, y_train8, y_test8 = train_test_split(X8, y, random_state=42, test_size=0.2, stratify=y)

In [49]:
X23 = df23.drop(columns=['id', 'flag'])

X_train23, X_test23, y_train23, y_test23 = train_test_split(X23, y, random_state=42, test_size=0.2, stratify=y)

In [50]:
print(X_train23.shape)
print(X_test23.shape)

(2400000, 420)
(600000, 420)


In [51]:
def train_model(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    predict_proba_train = model.predict_proba(X_train)[:,1]
    predict_proba_test = model.predict_proba(X_test)[:,1]
    predict_test = model.predict(X_test)
    print(roc_auc_score(y_train, predict_proba_train))
    print(roc_auc_score(y_test, predict_proba_test))
    print(precision_score(y_test, predict_test))
    print(recall_score(y_test, predict_test))
    print(confusion_matrix(y_test, predict_test))
    return predict_proba_test, predict_test, predict_proba_train

In [52]:
lgbm5 = LGBMClassifier(class_weight={0:1, 1:32}, n_jobs=-1, random_state=42, reg_lambda=1000, reg_alpha=0.4)

predict_proba_test_lgbm5, predict_test_lgbm5, predict_proba_train5 = train_model(X_train5, X_test5, y_train5, y_test5, lgbm5)

[LightGBM] [Info] Number of positive: 85154, number of negative: 2314846
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.250921 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2139
[LightGBM] [Info] Number of data points in the train set: 2400000, number of used features: 404
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.540685 -> initscore=0.163099
[LightGBM] [Info] Start training from score 0.163099
0.7630700240214419
0.7557876238786805
0.06708196908940038
0.7709037955655769
[[350482 228230]
 [  4877  16411]]


In [53]:
lgbm8 = LGBMClassifier(class_weight={0:1, 1:32}, n_jobs=-1, random_state=42, reg_lambda=1000, reg_alpha=0.4)

predict_proba_test_lgbm8, predict_test_lgbm8, predict_proba_train8 = train_model(X_train8, X_test8, y_train8, y_test8, lgbm8)

[LightGBM] [Info] Number of positive: 85154, number of negative: 2314846
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.583998 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2833
[LightGBM] [Info] Number of data points in the train set: 2400000, number of used features: 406
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.540685 -> initscore=0.163099
[LightGBM] [Info] Start training from score 0.163099
0.7655373310798825
0.7572988805086831
0.06713010152077302
0.7734404359263435
[[349907 228805]
 [  4823  16465]]


In [54]:
lgbm23 = LGBMClassifier(class_weight={0:1, 1:32}, n_jobs=-1, random_state=42, reg_lambda=1000, reg_alpha=0.4)

predict_proba_test_lgbm23, predict_test_lgbm23, predict_proba_train23 = train_model(X_train23, X_test23, y_train23, y_test23, lgbm23)

[LightGBM] [Info] Number of positive: 85154, number of negative: 2314846
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.862296 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4963
[LightGBM] [Info] Number of data points in the train set: 2400000, number of used features: 409
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.540685 -> initscore=0.163099
[LightGBM] [Info] Start training from score 0.163099
0.7660223118087348
0.7571226582458285
0.06682729814068392
0.7725948891394213
[[349047 229665]
 [  4841  16447]]


In [55]:
predict_proba_train5 = list(predict_proba_train5)
predict_proba_train8 = list(predict_proba_train8)
predict_proba_train23 = list(predict_proba_train23)
ensamble_predict_proba_train = pd.DataFrame([
    predict_proba_train5,
    predict_proba_train8,
    predict_proba_train23
]).mean().to_list()
roc_auc_score(y_train23, ensamble_predict_proba_train)

0.769800722667592

In [56]:
predict_proba_test_lgbm5 = list(predict_proba_test_lgbm5)
predict_proba_test_lgbm8 = list(predict_proba_test_lgbm8)
predict_proba_test_lgbm23 = list(predict_proba_test_lgbm23)
ensamble_predict_proba = pd.DataFrame([
    predict_proba_test_lgbm5,
    predict_proba_test_lgbm8,
    predict_proba_test_lgbm23
]).mean().to_list()
roc_auc_score(y_test23, ensamble_predict_proba)

0.7615495535417222

In [57]:
for i in range(0, len(ensamble_predict_proba)):
    if ensamble_predict_proba[i] < 0.5:
        ensamble_predict_proba[i] = 0
    else:
        ensamble_predict_proba[i] = 1
        
print(precision_score(y_test23, ensamble_predict_proba))
print(recall_score(y_test23, ensamble_predict_proba))
print(confusion_matrix(y_test23, ensamble_predict_proba))

0.0679409623838761
0.7760710259301015
[[352066 226646]
 [  4767  16521]]


In [ ]:
# Данные разбиты в соотношении 80/20. Выведена размерность тренировочной и тестовой выборок.
# Обучены модели и на основе их значений выведен результат из ансамбля предсказаний

# 5. Подготовьте автоматизированный пайплайн, который по вызову fit будет готовить данные и обучать модель, а по вызову predict — делать предсказания на заданном наборе данных. Обучите свой пайплайн подготовки данных и обучения модели и сохраните результат обучения в бинарном формате pickle.

In [5]:
def pipeline_processing(path):
    
    def dill_dump(path, model):
        with open(path, 'wb') as file:
            dill.dump(model, file)

    def dill_load(path):
        with open(path, 'rb') as file:
            model = dill.load(file)
        return model
    
    def agg_rn(df=pd.DataFrame) -> pd.DataFrame:
        
        '''Функция группирует по id признак rn и считает количество кредитов на 1 id'''
        
        df_agg_rn = df[['id', 'rn']].copy()
        df_agg_rn = df_agg_rn.groupby(['id']).agg('count').astype(np.int8).reset_index()
        return df_agg_rn
    
    def del_feature(df=pd.DataFrame, col=str) -> pd.DataFrame:
        
        '''Функция удаляет признак'''
        
        df_new = df.copy()
        return df_new.drop(columns=col)

    def invert(df=pd.DataFrame, columns=list, col_id=False) -> pd.DataFrame:
        
        '''Функция инвертирует значения выбранных признаков'''
        
        df_invert = df[['id'] + columns].copy()
        for col in columns:
            df_invert[col] = df_invert[col].apply(lambda x: 1 if x==0 else 0) #делаем реверс значений по каждой колонке
        if col_id == False:
            return df_invert.drop(columns='id')
        else:
            return df_invert
    
    def open_loans_credit(df=pd.DataFrame) -> pd.Series:
        
        '''Функция создает новый признак open_loans - количество открытых кредитов у одного id'''
        
        df_open = df[['id', 'is_zero_util']].copy()
        df_open = invert(df_open, ['is_zero_util'], col_id=True)
        df_open = df_open.groupby(['id']).agg('sum').astype(np.int8).reset_index()
        df_open = df_open.rename(columns={'is_zero_util':'open_loans'})
        return df_open['open_loans']
    
    def growth_limit_feature(df=pd.DataFrame) -> pd.Series:
        
        '''Функция берет первый и последний кредит клиента и считает разницу кредитного лимита,
        таким образом получаем новый признак growth_limit - изменение кредитного лимита.
        Если кредитный лимит уменьшился - значит у клиента не очень хорошо с финансами'''
        
        df_change = df[['id', 'pre_loans_credit_limit']].copy()
        df_change_first = df_change.drop_duplicates(subset=['id'], keep='first').rename(columns={
            'pre_loans_credit_limit': 'first'
        }).set_index('id')
        df_change_last = df_change.drop_duplicates(subset=['id'], keep='last').rename(columns={
            'pre_loans_credit_limit': 'last'
        }).set_index('id')
        df_change = pd.concat([df_change_first, df_change_last], axis=1).reset_index()
        df_change['growth_limit'] = df_change['last'] - df_change['first']
        return df_change['growth_limit']
    
    def convert_one_features(df0=pd.DataFrame, column=str, len_of_credit_history=int, col_id=False) -> pd.DataFrame:
        
        '''Функция преобразовывает один признак, указанный в column взяв при этом n-последних кредитов = 
        len_of_credit_history.'''
        
        df_col = df0[['id', column]]
    
        ohe = OneHotEncoder(sparse=False, dtype=np.int8)
        df_col = df_col.groupby(['id']).tail(len_of_credit_history).reset_index(drop=True) # выбор последних n-кредитов
        df_col[ohe.get_feature_names_out()] = ohe.fit_transform(df_col[[column]])
        df_col = df_col.drop(columns=column)
        df_col = df_col.groupby(['id']).agg('sum').reset_index()
        if col_id == False:
            df_col = df_col.drop(columns='id')
        return df_col
    
    def convert_df(df0=pd.DataFrame, len_of_credit_history=int, columns_output=None) -> pd.DataFrame:
        
        '''Функция собирает датасет из одного parquet файла'''
        
        df = pd.DataFrame()
    
        list_to_invert = [ 
            'is_zero_loans5',
            'is_zero_loans530',
            'is_zero_loans3060',
            'is_zero_loans6090',
            'is_zero_loans90'
        ]
    
        df[['id', 'rn']] = agg_rn(df0).astype(np.int32) # получаем новый агрегированный датафрейм по id с признаком rn - колич. кредитов
        df0[list_to_invert] = invert(df0, list_to_invert) # инвертируем необходимые признаки в переданном в функцию датасете
        df0 = del_feature(df0, 'pre_loans_total_overdue') # удаляем бесполезный признак в переданном в функцию датасете
        df['open_loans'] = open_loans_credit(df0) # добавляем к агрегированному датасету агрегированный признак - открытые кредиты
        df['growth_limit'] = growth_limit_feature(df0) # добавляем к агрегированному датасету агрегированный признак growth_limit
    
        columns = df0.columns.to_list()[2:] # так как id и rn уже сагрегированы, то обрезаим их из списка колонок
        for column in columns:
            df_one_columns = convert_one_features(df0, column, len_of_credit_history).astype(np.int8) # все фичи, кроме id и rn конвертируются в int8
            df = pd.concat([df, df_one_columns], axis=1)
            
        if columns_output != None:
            columns_df = list(df.columns)
            list_result = list((Counter(columns_output) - Counter(columns_df)).elements()) # ищем каких колонок не хватает
            df[list_result] = 0 # и добавляем в датасет нулевые значения
            df = df[columns_output] # убирает ошибку, что признаки передаются в модель не в том порядке
        
        try:
            df = df.drop(columns='id')
        except:
            pass

        return df
    
    def convert_parquet(path=str):

        '''Функция собирает итоговый датасет из parquet файлов и сохраняет в виде parquet'''

        files = [f for f in listdir(path) if isfile(join(path, f))] # создаем список файлов в директории

        df = pd.DataFrame()
        for file in files: # итерируем по списку, загружаем файл из списка
            df0 = pd.read_parquet(path + file)
            df0[df0.columns.to_list()[1:]] = df0[df0.columns.to_list()[1:]].astype(np.int8) # изменяем тип данных кроме id
            df = pd.concat([df, df0], axis=0) # объединяем файлы по строкам
        df = df.sort_values(by=['id', 'rn']) # отсортируем итоговый датасет, так как файлы загружаются не по порядку
        df.to_parquet(f'result_data/result.pq')
        print(f'Собран итоговый датасет и сохранен как result_data/result.pq')
            
    '''Соберем итоговый датасет из паркет файлов'''
    convert_parquet(path)
    
    '''Загружаем итоговый датасет и таргеты к нему'''
    
    df0 = pd.read_parquet('result_data/result.pq')
    target = pd.read_csv('train_target.csv').sort_values(by=['id'])
    
    '''Cконвертируем весь датасет целиком, чтобы получить список фичей, чтобы не возникало ошибок в Pipeline'''
    
    columns = list(convert_df(df0, len_of_credit_history=58).columns)
    print('Cписок признаков с последними 58 кредитами получен')
    
    '''Разделяем на train и test не саму выборку, так как она содержит дубликаты, а индексы файла tagret,
    и тогда вся выборка будет разделена корректно, если выбрать нужные индексы'''

    X = target.drop(columns='flag') # необходимы только индексы
    y = target['flag']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)
    X_train = df0.set_index('id').loc[X_train.index.to_list()].reset_index() # перезаписываем X_train данными из df0 c нужными id
    X_test = df0.set_index('id').loc[X_test.index.to_list()].reset_index() # перезаписываем X_test данными из df0 c нужными id
    y_train, y_test = y_train.sort_index().to_list(), y_test.sort_index().to_list()
    
    model1 = LGBMClassifier(class_weight={0:1, 1:32}, n_jobs=-1, random_state=42, reg_lambda=1000, reg_alpha=0.4)
    model2 = LGBMClassifier(class_weight={0:1, 1:32}, n_jobs=-1, random_state=42, reg_lambda=1000, reg_alpha=0.4)
    model3 = LGBMClassifier(class_weight={0:1, 1:32}, n_jobs=-1, random_state=42, reg_lambda=1000, reg_alpha=0.4)

    pipe5 = Pipeline(steps=[
        ('preprocessor', FunctionTransformer(convert_df, kw_args={'len_of_credit_history':5, 'columns_output':columns})),
        ('classifier', model1)
    ])

    pipe8 = Pipeline(steps=[
        ('preprocessor', FunctionTransformer(convert_df, kw_args={'len_of_credit_history':8, 'columns_output':columns})),
        ('classifier', model2)
    ])

    pipe23 = Pipeline(steps=[
        ('preprocessor', FunctionTransformer(convert_df, kw_args={'len_of_credit_history':23, 'columns_output':columns})),
        ('classifier', model3)
    ])


    class Pipes:
        def __init__(self, pipe1, pipe2, pipe3):
            self.pipe1 = pipe1
            self.pipe2 = pipe2
            self.pipe3 = pipe3

        def fit(self, X,y):
            self.pipe1.fit(X,y)
            print('Обучение 1 модели завершено')
            self.pipe2.fit(X,y)
            print('Обучение 2 модели завершено')
            self.pipe3.fit(X,y)
            print('Обучение 3 модели завершено')
                     
        def predict(self, test):
            pred1 = list(self.pipe1.predict_proba(test)[:,1])
            pred2 = list(self.pipe2.predict_proba(test)[:,1])
            pred3 = list(self.pipe3.predict_proba(test)[:,1])
            return pd.DataFrame([pred1,pred2,pred3]).mean().to_list()
    
    pipelines = Pipes(pipe5, pipe8, pipe23)
    pipelines.fit(X_train, y_train)
    print('Обучение моделей завершено')
    predict_proba_train = pipelines.predict(X_train)
    predict_proba_test = pipelines.predict(X_test)
    roc_test = roc_auc_score(y_test, predict_proba_test)
    roc_train = roc_auc_score(y_train, predict_proba_train)
    print(roc_train)
    print(roc_test)
    
    
    for i in range(0, len(predict_proba_test)):
        if predict_proba_test[i] < 0.5:
            predict_proba_test[i] = 0
        else:
            predict_proba_test[i] = 1

    print(precision_score(y_test, predict_proba_test))
    print(recall_score(y_test, predict_proba_test))
    print(confusion_matrix(y_test, predict_proba_test))
    
    pip_list = !pip list
    
    dict_final_model = {
        'model':pipelines,
        'roc_train':roc_train,
        'roc_test':roc_test,
        'version':1.0,
        'author': 'Sergey Savin',
        'list_of_libraries':pip_list,
        'date':'10/07/2024'
    }
    
    dill_dump('working/final_model.pkl', dict_final_model)

In [6]:
pipeline_processing('train_data/')

Собран итоговый датасет и сохранен как result_data/result.pq
Cписок признаков с последними 58 кредитами получен
[LightGBM] [Info] Number of positive: 85154, number of negative: 2314846
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 3.043938 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2141
[LightGBM] [Info] Number of data points in the train set: 2400000, number of used features: 405
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.540685 -> initscore=0.163099
[LightGBM] [Info] Start training from score 0.163099
Обучение 1 модели завершено
[LightGBM] [Info] Number of positive: 85154, number of negative: 2314846
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.818036 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


In [7]:
#  Проверим предсказывает ли обученный pipeline из pickle

def dill_load(path):
    with open(path, 'rb') as file:
        model = dill.load(file)
    return model

In [9]:
model = dill_load('working/final_model.pkl')

df0 = pd.read_parquet('result_data/result.pq') #df0 = pd.read_parquet(f'result_data/result.pq')
target = pd.read_csv('train_target.csv')

predict_proba = model['model'].predict(df0)
roc_auc_score(target['flag'].to_list(), predict_proba)

0.7092767115375811